In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("random_dataset.json")#json dosyası bu şekilde okunuyor.

In [4]:
print("Eksik veri sayısı:\n", df.isna().sum())

Eksik veri sayısı:
 product_id           0
customer_id          0
product_name         0
categories         115
price              102
purchaseDate        98
quantity            83
customers_score      0
dtype: int64


In [5]:
#CATEGORİES
#Kategori sutunlarının kaç kez tekrar ettiğini buluyoruz.
number_of_categories = df["categories"].value_counts(dropna=False)
print(number_of_categories)
#eksik verileri siliyoruz
df = df.dropna(subset=["categories"])

#PRİCE
#Her kategoriye ait ortalama alıp price null değerlerine atıyoruz.
df["price"] = df.groupby("categories")["price"].transform(lambda x: x.fillna(x.mean()))


#QUANTİTY
#Her kategoriye ait ortalama alıp quantity null değerlerine atıyoruz.
df["quantity"] = df.groupby("categories")["quantity"].transform(lambda x: x.fillna(x.mean()))


#DATE
#Belirli bir tarihte çok fazla alışveriş yapılmış mı kontrol ediyoruz.
print(df["purchaseDate"].value_counts())

#eksik verileri siliyoruz
df = df.dropna(subset=["purchaseDate"])

print(df.isna().sum()) 

categories
Electronics    443
Fashion        181
None           115
Furniture       96
Footwear        88
Accessories     77
Name: count, dtype: int64
purchaseDate
20/04/2022    3
2021/11/24    3
2023/07/11    3
11/01/2020    2
2024/05/11    2
             ..
2021/05/09    1
10-03-2023    1
2023/09/25    1
07-06-2024    1
05/01/2023    1
Name: count, Length: 745, dtype: int64
product_id         0
customer_id        0
product_name       0
categories         0
price              0
purchaseDate       0
quantity           0
customers_score    0
dtype: int64


In [6]:
# Tarih formatlarını uygun hale getirme
print(df["purchaseDate"])

# Farklı formatları belirleyerek tek tipe dönüştürme
def fix_date_format(date_str):
    for fmt in ("%d-%m-%Y", "%m-%d-%Y", "%d/%m/%Y","%Y/%m/%d","%Y-%m-%d"):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # Eğer format uymuyorsa NaT döndür


# Fonksiyonu veri setine uygula
df["purchaseDate"] = df["purchaseDate"].apply(fix_date_format)

# Tarihleri DD-MM-YYYY formatında string olarak sakla
df["purchaseDate"] = df["purchaseDate"].dt.strftime("%d-%m-%Y")

print(df["purchaseDate"])

#NaT olan veri var mı kontrolunu sağladık.
print(df["purchaseDate"].isna)


0      02-12-2020
2      27/05/2022
3      01-17-2021
4      21/04/2021
5      01-24-2023
          ...    
994    16/05/2022
995    2021/07/26
996    2020-02-07
997    2020-10-14
998    05/01/2023
Name: purchaseDate, Length: 797, dtype: object
0      02-12-2020
2      27-05-2022
3      17-01-2021
4      21-04-2021
5      24-01-2023
          ...    
994    16-05-2022
995    26-07-2021
996    07-02-2020
997    14-10-2020
998    05-01-2023
Name: purchaseDate, Length: 797, dtype: object
<bound method Series.isna of 0      02-12-2020
2      27-05-2022
3      17-01-2021
4      21-04-2021
5      24-01-2023
          ...    
994    16-05-2022
995    26-07-2021
996    07-02-2020
997    14-10-2020
998    05-01-2023
Name: purchaseDate, Length: 797, dtype: object>


In [20]:
df["best_score"] = df["quantity"] * df["customers_score"]
best_product = df.groupby("product_name")["best_score"].sum().idxmax()
print(f"En popüler ürün (Satış ve Memnuniyete Göre): {best_product}")

df[df["product_name"] == "Desk Chair"]
df.loc[df["product_name"] == "Desk Chair", "price"] *= 1.10


En popüler ürün (Satış ve Memnuniyete Göre): Desk Chair


In [ ]:
df[df["product_name"] == "Desk Chair"]

In [ ]:
df.loc[df["product_name"] == "Desk Chair", "price"] *= 1.10

In [21]:
#3
top_selling_products = df.groupby("product_name")["quantity"].sum().sort_values(ascending=False)
print(top_selling_products.head(10))  # En çok satılan 10 ürünü göster


product_name
                  247.689199
Sunglasses        234.000000
Headphones        227.682927
Running Shoes     223.819277
Smartphone        220.682927
Tablet            212.492683
Gaming Console    208.556098
Desk Chair        202.241379
Wristwatch        184.428571
Laptop            180.936585
Name: quantity, dtype: float64


In [22]:
correlation = df[["price", "quantity"]].corr()
print(correlation)


             price  quantity
price     1.000000 -0.035924
quantity -0.035924  1.000000


In [24]:
average_price_per_category = df.groupby("categories")["price"].mean()
print(average_price_per_category)


categories
Accessories     970.439572
Electronics    1025.546562
Fashion         931.688341
Footwear       1026.338953
Furniture      1039.164555
Name: price, dtype: float64


In [ ]:
df["purchaseDate"] = pd.to_datetime(df["purchaseDate"], dayfirst=True)  # Tarih formatını düzelt
filtered_data = df[df["purchaseDate"].dt.year == 2024]  # 2024 yılını filtrele
top_products_2024 = filtered_data.groupby("product_name")["quantity"].sum().sort_values(ascending=False)
print(top_products_2024.head(10))  # En çok satılan 10 ürünü göster


product_name
Sunglasses        74.857143
Laptop            58.936585
Headphones        58.873171
Smartphone        57.873171
Desk Chair        51.000000
Running Shoes     47.000000
Tablet            43.873171
Wristwatch        43.714286
Gaming Console    39.873171
                  36.873171
Name: quantity, dtype: float64


In [ ]:
df["total_expenditure"] = df["price"] * df["quantity"]  # Toplam harcama hesapla

def total_expenditure(x):
    if x < 500:
        return "low"
    elif x < 2000:
        return "middle"
    else:
        return "high"

df["spending_level"] = df["total_expenditure"].apply(total_expenditure)
spending_grups = df.groupby("spending_level")["product_id"].nunique()  # Kaç müşteri olduğunu say
print(spending_grups)


spending_level
Düşük      77
Orta      279
Yüksek    441
Name: product_id, dtype: int64


In [30]:
#4
average_price_per_category = df.groupby("categories")["price"].mean()
print(average_price_per_category)


categories
Accessories     970.439572
Electronics    1025.546562
Fashion         931.688341
Footwear       1026.338953
Furniture      1039.164555
Name: price, dtype: float64


In [52]:
def price_status(row, average_price_per_category):
    category_average_price = average_price_per_category.get(row["categories"], 0)
    if category_average_price == 0:  # Kategorinin ortalama fiyatı bulunamazsa
        return "null"
    if row["price"] > category_average_price * 2:
        return "extremely expensive"
    elif row["price"] < category_average_price / 2:
        return "extremely cheap"
    else:
        return "ordinary"

df["price_status"] = df.apply(price_status, axis=1, average_price_per_category=average_price_per_category)


In [53]:
print(df[[ "price", "price_status"]].head(10))


          price         price_status
0   1026.914253             ordinary
2   1351.160000             ordinary
3   1605.220000             ordinary
4    877.020000             ordinary
5           NaN             ordinary
6    673.700000             ordinary
7   1576.950000             ordinary
8           NaN             ordinary
9   2150.599000  extremely expensive
10  1026.914253             ordinary


In [54]:
# "Aşırı Ucuz" olan ürünleri kategori ortalamasına eşitleme ve NaN değerleri kontrol etme
df.loc[df["price_status"] == "extremely cheap", "price"] = df.loc[df["price_status"] == "extremely cheap"].apply(
    lambda row: average_price_per_category.get(row["categories"], row["price"]) if pd.isna(row["price"]) or row["price"] == "extremely cheap" 
    else row["price"], axis=1)


In [55]:
print(df[[ "price", "price_status"]].head(10))


          price         price_status
0   1026.914253             ordinary
2   1351.160000             ordinary
3   1605.220000             ordinary
4    877.020000             ordinary
5           NaN             ordinary
6    673.700000             ordinary
7   1576.950000             ordinary
8           NaN             ordinary
9   2150.599000  extremely expensive
10  1026.914253             ordinary
